In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# from ultralytics import YOLO
from pyproj import Proj, transform
from random import randint, random, choice

In [3]:
# model = YOLO('../best.pt')

#### Задаем параметры камеры

In [17]:
wgs84 = Proj(init='epsg:4326')

d:\MyProgramms\Programs\Python\pothole-coordinates\.venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [18]:
resolution = [i for i in range(400, 4000, 200)]
coefs = [i for i in range(10, 10000, 200)]

class Camera():
    def __init__(self):
        self.updateParam()
    
    def updateParam(self):
        self.height = randint(5, 30)
        self.azimut = randint(0, 360)
        self.angleView = randint(50, 160)
        self.resolition = np.array([choice(resolution), choice(resolution)])
        self.worldCoords = np.random.rand(2) * np.array([choice(coefs), choice(coefs)])

#### Высчитываем мировые координаты ямы 

In [21]:
def pixelsToMetresOffset(cam: Camera, pothole_imgCoords: np.array, proj: Proj):
    quartMatrix = np.array([[np.cos(cam.azimut),-np.sin(cam.azimut)],
                             [np.sin(cam.azimut),np.cos(cam.azimut)]]) # Формирование матрицы поворота

    lon, lat = cam.worldCoords

    D = (pothole_imgCoords - cam.resolition / 2) / (cam.resolition / 2) #Вычисление вектора местонахождения ямы
    vector = D * cam.height * np.tan(cam.angleView/2) #Преобразование вектора в реальные размеры (в метрах)

    dx, dy = np.matmul(vector, quartMatrix) #Умножение на матрицу поворота для определения реального смещения

    new_lon, new_lat = transform(proj, proj, lon + dx / (proj.crs.ellipsoid.semi_major_metre * np.cos(lat * np.pi / 180.0)),
                                        lat + dy / proj.crs.ellipsoid.semi_major_metre) #Вычисление мировых координат ямы
    return new_lon, new_lat

In [22]:
cam = Camera()
potholeImgCoords = np.random.randint([0,0], [cam.resolition[0], cam.resolition[1]])
potholeWorldCoords = pixelsToMetresOffset(cam, potholeImgCoords, wgs84)
print(potholeImgCoords, potholeWorldCoords)

[1282 2197] (582.1444598138287, 6797.39270207796)


C:\Users\sanya\AppData\Local\Temp\ipykernel_13508\994254206.py:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_lon, new_lat = transform(proj, proj, lon + dx / (proj.crs.ellipsoid.semi_major_metre * np.cos(lat * np.pi / 180.0)),
